In [1]:
import pandas as pd
import requests

In [2]:
symbol = "EMAAR"
exchange = "DFM"  # ADSM or DFM, ADSM = ADX
session_id = "e23b8b7e-5e52-4ce8-ade6-0c4e3311b720"
start = "1900-01-31T20:18:48.000Z"
end = "2100-01-31T20:00:00.000Z"
interval = "1"  # 1 days
additional_slug = "1753968492999"

url = f"https://mobile.intlsecurities.ae/FITDataFeedServiceGateway/DataFeedService.asmx/datafeedDFN?session={session_id}&symbol={symbol},{exchange}&period=day&from={start}&to={end}&interval={1}${additional_slug}"

In [3]:
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        df = pd.DataFrame(data)

        if "DT" in df.columns:
            df.rename(columns={"DT": "Date"}, inplace=True)
            df["Date"] = pd.to_datetime(df["Date"]).dt.date
            df = df.set_index("Date")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    except ValueError:
        print("Error: Could not decode JSON from the response.")

    return df, data


df, data = fetch_data(url)
print(data[0:5])
print(df.head())

[{'DT': '2025-10-07T00:00:00.000Z', 'Open': 13.55, 'High': 13.65, 'Low': 13.55, 'Close': 13.55, 'Volume': 3714386}, {'DT': '2025-10-06T00:00:00.000Z', 'Open': 13.65, 'High': 13.75, 'Low': 13.55, 'Close': 13.6, 'Volume': 6727653}, {'DT': '2025-10-03T00:00:00.000Z', 'Open': 13.65, 'High': 13.65, 'Low': 13.45, 'Close': 13.6, 'Volume': 6876490}, {'DT': '2025-10-02T00:00:00.000Z', 'Open': 13.45, 'High': 13.75, 'Low': 13.4, 'Close': 13.65, 'Volume': 18133166}, {'DT': '2025-10-01T00:00:00.000Z', 'Open': 13.1, 'High': 13.45, 'Low': 13.05, 'Close': 13.4, 'Volume': 13081448}]
             Open   High    Low  Close    Volume
Date                                            
2025-10-07  13.55  13.65  13.55  13.55   3714386
2025-10-06  13.65  13.75  13.55  13.60   6727653
2025-10-03  13.65  13.65  13.45  13.60   6876490
2025-10-02  13.45  13.75  13.40  13.65  18133166
2025-10-01  13.10  13.45  13.05  13.40  13081448


In [4]:
trans_log = pd.read_csv("raw-data/IntlSecurities_July2025.csv")
symbol_market_pair = trans_log[["Symbol", "Exchange"]].dropna().drop_duplicates()
symbol_market_pair

,Symbol,Exchange
1,ALDAR,ADX
2,EMAAR,DFM
3,CHADX15,ADX
4,BURJEEL,ADX
5,MULTIPLY,ADX
6,IHC,ADX
8,AMR,ADX
9,ASM,ADX


In [5]:
for _, row in symbol_market_pair.iterrows():
    symbol = row["Symbol"]
    exchange = row["Exchange"] if row["Exchange"] == "DFM" else "ADSM"
    url = f"https://mobile.intlsecurities.ae/FITDataFeedServiceGateway/DataFeedService.asmx/datafeedDFN?session={session_id}&symbol={symbol},{exchange}&period=day&from={start}&to={end}&interval={1}${additional_slug}"
    df, _ = fetch_data(url)
    df.reset_index().to_csv(f"../data/manual-source/prices/{symbol}.csv", index=False)

![test.png](IntlSecurities-data-scrapping.png)